# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("../output_files/city.csv")
file.dropna()
file.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,New Norfolk,55,AU,1569425354,66,-42.78,147.06,50.00,11.41
1,1,Isiro,100,CD,1569425356,78,2.77,27.62,78.93,3.24
2,2,Torbay,90,CA,1569425355,100,47.66,-52.73,50.00,17.22
3,3,Susanville,1,US,1569425356,76,40.42,-120.65,57.99,3.36
4,4,Walvis Bay,0,NaN,1569425356,68,-22.95,14.51,66.20,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [9]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,Rikitea,0,PF,1569425361,67,-23.12,-134.97,71.01,4.90
29,29,Tres Arroyos,0,AR,1569425363,40,-38.38,-60.28,71.55,6.35
37,37,Hermanus,0,ZA,1569425366,44,-34.42,19.24,77.00,10.00
98,98,Miyako,0,JP,1569425382,78,33.63,130.94,71.60,8.05
108,108,Paso de Carrasco,0,UY,1569425385,31,-34.86,-56.05,77.00,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -23.1276524,
                    "lng": -134.9656596
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.12637127010728,
                        "lng": -134.9643498701072
                    },
                    "southwest": {
                        "lat": -23.12907092989272,
                        "lng": -134.9670495298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "45da19509c7981427f6f1e9583a48b16fcc0c451",
            "name": "Pension Bianca & Benoit",
            "photos": [
                {
                    "height": 1536,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/11515043980962209

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -38.3693062,
                    "lng": -60.29775069999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -38.36800572010728,
                        "lng": -60.29633492010727
                    },
                    "southwest": {
                        "lat": -38.37070537989273,
                        "lng": -60.29903457989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "0f366c6d0df67a14e987d543c3ce1a081fe4b783",
            "name": "Elegance Hotel Tres Arroyos",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 3096,
                    "html_attributions": [


{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAAm8Mej7jbG2W1rmcpb5YwoBchP6qZIjINzVIuRwGrTW4UrhcsNjU5NxRvN2cvVN-DhCvbo71S7W9Kx4sAnYJDVn10MxmnPOIDQoW9S9POeZno_jdglmrPUfZrAQk6IbJ5cQJXlKsFixnmaGe3FoPRStuaYmDc8uNQCHe8h4cOLOgz2WqySTQUviEnqs12oBmV2H4fPSvobOQS5HG97kk84wYjR_xEBNAIi6O3vhdUcb8R9X8Z24WUEijCqI-uVxUEZOqxwPRnzRMThyUjGXiHBtjiwbn3hnMAPe4NxYPpKFahBQdf2Iq-XorsjVUi2kfNUtFsBD0bGntwOzaRMpB4aIRwkC_tqMI4MuQ-wTkiiaue41m317HJbxB_WTf8NW9J2FKVvy65gDHVrHuzBlscMSEG4r9VZ0H3hdMhVbq9GUVHMaFIXuFKrcLjH1imHDzj_B7gva09Sd",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.126115,
                    "lng": 18.834302
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.12476462010727,
                        "lng": 18.83565232989272
                    },
                    "southwest": {
                        "lat": -34.12746427989272,
                      

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAMf5esexkyGVUuOyb6UkmeTm2WU07-eWw-MKtAXQedbFs7G3jF8Ytfa1G4n-DYoDlQGY-fJLFfrfMLv-yFP6idFJHM08FmFD3uj_2qIHE3t7u8LKO9mbe1aLoPWBtdLi2msqqc0ixZVfrzLnB8aiwUZqgIZVRxDazqT4B0uvhl366uam4DUf7IisNsMBS-MgwNLrTCWf6eek7RIPpDuSVikxtXUuykX0SDTq8MPTjjAbhXFXQNs4nPwESet0v20f1I7YYLVbgmoxmsyPAKvIrc1hygCiu-fdaQjbzbzB_vt_UIBqL522wBDOGnBkQnaBj9P1xelwAYCnxFIgwfxbGmBPYfkB142pXPLLo3P7cLWIv3jEDUtBSHuHAvlQVK6kbNDNjjQxckuodf-xPNwiaO8SEOVVPsET_PXz5OL5DdunYJkaFDbuuenN-83s0cak3EyjhqnD6MSH",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 33.3705667,
                    "lng": 130.5297096
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.37204792989272,
                        "lng": 130.5310582298927
                    },
                    "southwest": {
                        "lat": 33.36934827010728,
                      

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAP42xSNx0YJjC6gYBDmS340PZwtxuAj9V2NBbOUB8c1njRe-WRtGrXE5_AmVscRPitbiAhYeZytB-2lpwQNPjDwP8_bbvD4Z18z4hVcp0V2yalx8HBpiHRHF_yeXhXEZIDem2kveB3g_h0pO4YwlZv1gPvmKkCX6xo63H9o7tcELle1sRAx4qCykLD9tn6e_NRzj4v3eO6i53De5jN3vFlZU6YVj-XyWcB3eFi1XoIBxxrWIyZNM-gwz8S0EknjNRiGB9YvibvmYkt57sUFqrTMtR60EfcWlfJoPhvYl6ajXgv99JniglBYls02v9JQZDVHztnBCS3RvUBw-Lj84WYWgdZfurv-HlAHI6eW_3pS7slPi5riJwdmmQXBdquLzTxEKtslZDnMSOWFGzJVTD08SEEz_3rvXrRd7uQdGtjREyg4aFBoXk5U-jjyyI5kEEGvZzYKA4nLH",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.9062142,
                    "lng": -56.1901277
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.90486647010728,
                        "lng": -56.18881072010728
                    },
                    "southwest": {
                        "lat": -34.90756612989273,
                  

...

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -19.5369757,
                    "lng": 45.4596401
                },
                "viewport": {
                    "northeast": {
                        "lat": -19.53561237010728,
                        "lng": 45.46091542989273
                    },
                    "southwest": {
                        "lat": -19.53831202989272,
                        "lng": 45.45821577010729
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "9c58aefa5e7cba557b55caaee309a7fd508599ce",
            "name": "H\u00f4tel Arc En Ciel Miandrivazo",
            "photos": [
                {
                    "height": 2268,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/1084414106

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAMe8zpLPKZDv76lnkpwstbnMnIdjlA0ZXa-3zXYi1FYbnTR-0fIrw4bfz9bzl1_Ix9RnQfol3X033pOHAc5Xx8yvSqeuI-76LrZNbjlsC0KiXfNqw2D3Qg9BHPjY50p8qCesr91dFt8yPCj7u_BuomZZ0NSmFzS4BcadXT4j6uFr6Jf5Zil1MvHUUpVsB7JWaFVy7OD6oQ2jC-5HNUGv8ro3Y0UF5nDYkvxIbwMCHqOMqYAorOG22YVHmAkfneWUwdZdQUbejpicI5sGKbFIOCJlCxpAUbOUACff3JwmNmJECRQAz5td4RMb4e7gLNXNCq2QQ72zcBBBB4q99KsAmVSUHc1YDvZmHPlqpMEnU_kkNesVe0QRW37LehFONMeOY40pxU1rttdxTw4E8esRMxISEMiQbkf4WNxXz-_f4S1DPF8aFHUyr6--kbx9CVvbo82ORxTMUWgw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -8.7514413,
                    "lng": 121.8584476
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.750380120107279,
                        "lng": 121.8597848798927
                    },
                    "southwest": {
                        "lat": -8.753079779892722,
                    

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAHZh147Pr5q4NlXRkB23uepiDetiU5PzZ3fZlzbsPGwiumY_zQ6wTlIq_4syqS7UuOM2ClzwrEspWfAxhfppWfUrFswYJU2nmaFVAb5VrKm9Ifw0mhCQJ_RnomU-CkNQ31AQFUmYwTLTIo-_C4Aqb2wPS8WpJocedTDWXVrHdkmOxK-cDlNP90gXd0l3c8Vw6FnYAX8JILcA2zqLnLfciD1n5zYpnSGAGfb6ws--7ek77VDYVeiDFfK2UNmsAHvs3hJcVV-eOclbDfjyPLVZJROTlbvJDoBXYo8KDhUhGBtEyT_b_LWVLK2i792bv8OUT2cVmGF8jxoSYH4ryyJmicEPoLVshjsVZe6_9-aB_HTYP-Iwh2k4ierswx1kaJ8L6Vt8rFT-ZI38kUhCiQFeqVUSEC6hhJDmgCulYTpMDkAm4J8aFOHBGotCxCP7_V36Nh4urodiZBGn",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 39.5231524,
                    "lng": 46.0262022
                },
                "viewport": {
                    "northeast": {
                        "lat": 39.52453222989272,
                        "lng": 46.02750422989272
                    },
                    "southwest": {
                        "lat": 39.52183257010728,
                       

{
    "html_attributions": [],
    "next_page_token": "CrQCJAEAAHa1vA7aS1ExPB4zGJBYr8SMQbc6iQzXfMNRr-T58yeYBKuLXqeB68evYdYW0f9T-655pEwZA1fZ7JuRCVVwHDT_MmO8H7JUZ2ozWXIhbQ0G1-H6iY2GE-BrvN6E8KFWpSDmPqFME_zpDJpJJbeG8Tv9KlALmHdrQfxSf-EHWw89E2EkmBDe8aDrW1O-DTrxbajE6Ygpb3HA7u2RzmiNXr-BOsVd5DN89Q_Yv3vYjY1aO52g1cO-0X86W4T_Z4-DApEEA-IZrxLMtUAye6b8cx7C11iPj_41I1XSvFfaSttUObymrDJJsfgKh15dEGVanPFF-5JnWZPTIWMnoxVRZtmgxLqWoLjeEvYc-bjpAJonBMxM06qQScQxg2lgZ9uNayzutzLruIuXHqeHIc8VrioSEG7lTTMQTHKpKrZ0zy5dM_AaFGHxumL_Y1gNsFCrKI7AxMoJfb84",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 42.829182,
                    "lng": 93.516948
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.83048082989272,
                        "lng": 93.51837912989272
                    },
                    "southwest": {
                        "lat": 42.82778117010728,
                        "

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,Rikitea,0,PF,1569425361,67,-23.12,-134.97,71.01,4.90,Pension Bianca & Benoit
1,29,Tres Arroyos,0,AR,1569425363,40,-38.38,-60.28,71.55,6.35,Elegance Hotel Tres Arroyos
2,37,Hermanus,0,ZA,1569425366,44,-34.42,19.24,77.00,10.00,Ocean Breeze Hotel & Conference Centre
3,98,Miyako,0,JP,1569425382,78,33.63,130.94,71.60,8.05,ホテル ビアントス
4,108,Paso de Carrasco,0,UY,1569425385,31,-34.86,-56.05,77.00,9.17,Hotel Balfer
5,117,Namibe,0,AO,1569425388,88,-15.19,12.15,70.47,9.48,Hotel ibis Styles Iu Namibe
6,118,Barrow,0,AR,1569425389,40,-38.31,-60.23,71.55,6.35,Elegance Hotel Tres Arroyos
7,136,Sayat,0,TM,1569425393,33,38.78,63.88,77.00,2.24,Jeyhun Oteli
8,183,Noumea,0,NC,1569425125,82,-22.28,166.46,73.99,2.24,Hotel Beaurivage
9,210,Tiznit,0,MA,1569425325,81,29.70,-9.73,70.47,4.34,Hôtel Aglou Beach


In [13]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))